In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import loadmat
import torch

In [ ]:
from geomechy.base import ItemDict, Properties 
from geomechy.base import BaseConstitutive, BaseElement, BaseLinearSolver
from geomechy.utils import lanczos, arnoldi

# Test Base Rotines

## 1. ItemDict

In [ ]:
item = ItemDict()
item.add(1,[1,2,3])
item.add(2,[3,4,5])
item.add(3,[6,7,8])
assert isinstance(item,dict)
assert item.get(1) == [1,2,3]
assert item.getData([1,2]) == {1: [1, 2, 3], 2: [3, 4, 5]}
print(item)

{1: [1, 2, 3], 2: [3, 4, 5], 3: [6, 7, 8]}


## 2. Properties

In [ ]:
props = Properties({"E":500, "nu":0.3})

assert props.E == 500
assert props.nu == 0.3
props.store("G",200)
assert props.G == 200
assert next(props.__iter__()) == ('E', 500)
print(props)

Attribute: E
500
Attribute: G
200
Attribute: nu
0.3



## 3. Base Constitutive

In [ ]:
params = {"E": {"dim": "Pascal","value": 500},"nu":{"dim":"dimless","value": 0.3}}

consti = BaseConstitutive(params)
assert consti.E == 500
assert consti.nu == 0.3
assert consti.current == {'i': 0}
assert consti.iIter == -1
consti.setCurrentParameter("G", 200)
assert consti.getCurrentParameter("G")  == 200
consti.setCurrentParameter("K", 300)
assert consti.getCurrentParameter("K")  == 300
consti.commitHistory()
assert consti.iIter == 0
assert consti.current == {'i': 0.0, 'G': 0.0, 'K': 0.0}
assert isinstance(consti.history,pd.DataFrame)
assert consti.getHistoryParameter("K",0) == 300
print(consti)

current iter: 0


## 4. BaseElement

## 5. Base Linear System Solver

In [ ]:
A  = csr_matrix([[5,0,0],[0,4,0],[0,0,7]])
b  = np.random.randn(3,1)
xo = np.random.randn(3,1)
tol = 1e-10
maxIter = 100

solver = BaseLinearSolver(A, b, xo, tol, maxIter)
assert solver.device == torch.device(type='cuda')
assert np.allclose(np.linalg.norm(solver.A.cpu().numpy()),np.linalg.norm(A.todense()))
assert np.allclose(np.linalg.norm(solver.b.cpu()),np.linalg.norm(b))
assert np.allclose(np.linalg.norm(solver.xo.cpu()),np.linalg.norm(xo))
assert np.allclose(solver.tol,tol)
assert np.allclose(solver.maxIter,maxIter)
assert hasattr(solver, 'A_sparsity')
assert hasattr(solver, 'b_sparsity')
assert np.allclose(solver.A_sparsity,6./9)
assert solver.b_sparsity == 0
print(solver)

Solver device: cuda


# Test Utils Routines

## 1. Lanczos Factorization

In [ ]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
A  = torch.from_numpy(loadmat('./assets/matlab/ACG.mat')["ACG"].toarray()).type(torch.float64).to(device)
b  = torch.from_numpy(loadmat('./assets/matlab/b.mat')["b"]).type(torch.float64).to(device)
xo = torch.from_numpy(loadmat('./assets/matlab/xo.mat')["xo"]).type(torch.float64).to(device)
r  = b - torch.matmul(A,xo)

Q_gpu, T_gpu = lanczos(A,r,m=100,reorthog=0)
A, Q, T = A.cpu().numpy(), Q_gpu.cpu().numpy(), T_gpu.cpu().numpy()
assert np.allclose(np.linalg.norm(np.matmul(A,Q[:,:-1])),np.linalg.norm(np.matmul(Q,T)))

## 2. Arnoldi Factorization

In [ ]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
A  = torch.from_numpy(loadmat('./assets/matlab/ACG.mat')["ACG"].toarray()).type(torch.float64).to(device)
b  = torch.from_numpy(loadmat('./assets/matlab/b.mat')["b"]).type(torch.float64).to(device)
xo = torch.from_numpy(loadmat('./assets/matlab/xo.mat')["xo"]).type(torch.float64).to(device)
r  = b - torch.matmul(A,xo)

Q_gpu, H_gpu, m = arnoldi(A,r,m=100)
A, Q, H = A.cpu().numpy(), Q_gpu.cpu().numpy(), H_gpu.cpu().numpy()
assert np.allclose(np.linalg.norm(np.matmul(A,Q[:,:-1])),np.linalg.norm(np.matmul(Q,H)))